# Train a gesture recognition model for microcontroller use

This notebook demonstrates how to train a 20kb gesture recognition model for [TensorFlow Lite for Microcontrollers](https://tensorflow.org/lite/microcontrollers/overview). It will produce the same model used in the [magic_wand](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/micro/examples/magic_wand) example application.

The model is designed to be used with [Google Colaboratory](https://colab.research.google.com).

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/magic_wand/train/train_magic_wand_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/magic_wand/train/train_magic_wand_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


Training is much faster using GPU acceleration. Before you proceed, ensure you are using a GPU runtime by going to **Runtime -> Change runtime type** and selecting **GPU**. Training will take around 5 minutes on a GPU runtime.

## Configure dependencies

Run the following cell to ensure the correct version of TensorFlow is used.

In [1]:
%tensorflow_version 2.x


We'll also clone the TensorFlow repository, which contains the training scripts, and copy them into our workspace.

In [2]:
!cd /content/
!rm -rf magic_wand_capture
!rm -rf train
# Clone the repository from GitHub
!git clone --depth 1 -q https://github.com/petewarden/magic_wand_capture
# Copy the training scripts into our workspace
!cp -r magic_wand_capture/train train

## Prepare the data

Next, we'll download the data and extract it into the expected location within the training scripts' directory.

In [13]:

# The scripts must be run from within the train directory
%cd /content/train

# Download the data we will use to train the model
!wget 'http://download.tensorflow.org/models/tflite/magic_wand/gesture_data_v2_2020_03_22.tar.gz'
# Extract the data into the train directory
!rm -rf gesture_data_v2
!tar xvzf gesture_data_v2_2020_03_22.tar.gz 1>/dev/null

/content/train
--2020-12-12 23:13:28--  http://download.tensorflow.org/models/tflite/magic_wand/gesture_data_v2_2020_03_22.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.217.128, 2607:f8b0:400c:c13::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.217.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278118 (272K) [application/gzip]
Saving to: ‘gesture_data_v2_2020_03_22.tar.gz.1’

gesture_data_v2_202 100%[===================>] 271.60K  --.-KB/s    in 0.001s  

2020-12-12 23:13:28 (404 MB/s) - ‘gesture_data_v2_2020_03_22.tar.gz.1’ saved [278118/278118]



We'll then run the scripts that split the data into training, validation, and test sets.

In [14]:
!mv gesture_data_v2/petewarden* . # good
!mv gesture_data_v2/aritrab* . # good
!mv gesture_data_v2/aybuke* . # good
!mv gesture_data_v2/imellado* . # good
!mv gesture_data_v2/jheez* . # good
!mv gesture_data_v2/oscarazu* . # good
!mv gesture_data_v2/owoowoo* . # good
#!mv gesture_data_v2/realmistic* . # bad
#!mv gesture_data_v2/stevenchun* . # bad
#!mv gesture_data_v2/vladionescu* . # bad

!rm -rf gesture_data_v2/*

!mv petewarden* gesture_data_v2/
!mv aritrab* gesture_data_v2/
!mv aybuke* gesture_data_v2/
!mv imellado* gesture_data_v2/
!mv jheez* gesture_data_v2/
!mv oscarazu* gesture_data_v2/
!mv owoowoo* gesture_data_v2/
#!mv realmistic* gesture_data_v2/
#!mv stevenchun* gesture_data_v2/
#!mv vladionescu* gesture_data_v2/


In [ ]:
# The scripts must be run from within the train directory
%cd /content/train

# Prepare the data
!python data_prepare.py
!head gesture_data_v2/all_data.json
!python data_split.py


## Load TensorBoard

Now, we set up TensorBoard so that we can graph our accuracy and loss as training proceeds.

In [ ]:
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs/scalars

# Visualize Training Data

Let's graph some of the data that we'll be using to train the model, to help us better understand it.

In [7]:
import math

def vector_magnitude(vec):
  x = vec[0]
  y = vec[1]
  z = vec[2]
  return math.sqrt((x * x) + (y * y) + (z * z))

def normalize_vector(vec):
  magnitude = vector_magnitude(vec)
  x = vec[0]
  y = vec[1]
  z = vec[2]
  normalized_x = x / magnitude
  normalized_y = y / magnitude
  normalized_z = z / magnitude
  return (normalized_x, normalized_y, normalized_z)  

def dot_product(a, b):
  return (a[0] * b[0], a[1] * b[1], a[2] * b[2])

def estimate_gravity_direction(sequence):
  x_total = 0.0
  y_total = 0.0
  z_total = 0.0
  for entry in sequence:
    x = entry[0]
    y = entry[1]
    z = entry[2]
    x_total += x
    y_total += y
    z_total += z
  return (x_total / len(sequence), y_total / len(sequence), z_total / len(sequence))

def remove_gravity_from_acceleration_data(sequence):
  gravity_direction = estimate_gravity_direction(sequence)
  gravity_x = gravity_direction[0] * 1.0
  gravity_y = gravity_direction[1] * 1.0
  gravity_z = gravity_direction[2] * 1.0
  result = []
  for entry in sequence:
    x = entry[0]
    y = entry[1]
    z = entry[2]
    normalized_x = x - gravity_x
    normalized_y = y - gravity_y
    normalized_z = z - gravity_z
    result.append([normalized_x, normalized_y, normalized_z])
  return result

def acceleration_data_into_positions(sequence):
  current_x = 0.0
  current_y = 0.0
  current_z = 0.0
  result = []
  for entry in sequence:
    x_velocity = entry[0]
    y_velocity = entry[1]
    z_velocity = entry[2]
    current_x += x_velocity
    current_y += y_velocity
    current_z += z_velocity
    result.append([current_x, current_y, current_z])
  return result


In [8]:
import data_load

data_loader = data_load.DataLoader("gesture_data_v2/training_data.json",
        "gesture_data_v2/validation_data.json",
        "gesture_data_v2/testing_data.json", 128)


train_data_length:6840
valid_data_length:75
test_data_length:80


In [9]:
import matplotlib.pyplot as plt
import json
import data_prepare
import ipywidgets as widgets
import os.path as path

def plot_data(acceleration_data, label):
  gravity_direction = estimate_gravity_direction(acceleration_data)
  gravity_free_data = remove_gravity_from_acceleration_data(acceleration_data)
  position_data = acceleration_data_into_positions(gravity_free_data)

  x_array = []
  y_array = []
  z_array = []
  for coords in position_data:
    x_array.append(coords[0])
    y_array.append(coords[1])
    z_array.append(coords[2])

  fig = plt.figure(figsize=(12.8, 4.8))
  fig.suptitle(label)

  ax = fig.add_subplot(131)
  ax.set_xlabel('x')
  ax.set_ylabel('y')
  ax.set_xlim(-20000, 20000)
  ax.set_ylim(-25000, 25000)
  ax.plot(x_array, y_array)

  ax = fig.add_subplot(132)
  ax.set_xlabel('x')
  ax.set_ylabel('z')
  ax.set_xlim(-20000, 20000)
  ax.set_ylim(-25000, 25000)
  ax.plot(x_array, z_array)

  ax = fig.add_subplot(133)
  ax.set_xlabel('y')
  ax.set_ylabel('z')
  ax.set_xlim(-20000, 20000)
  ax.set_ylim(-25000, 25000)
  ax.plot(y_array, z_array)

  plt.show()

hand_labels = []
labeled_ids = {}
if path.exists("hand_labels.json"):
  with open("hand_labels.json", "r") as file:
    for line in file:
      dic = json.loads(line)
      hand_labels.append(dic)
      labeled_ids[dic["source_id"]] = True

gesture_data = []
with open("gesture_data_v2/all_data.json", "r") as file:
  for index, line in enumerate(file):
    dic = json.loads(line)
    gesture_data.append(dic)
    example_data = dic[data_load.DATA_NAME]
    label = dic[data_load.LABEL_NAME]
    source_id = dic["source_id"]
    if source_id in labeled_ids or "hand_label" in dic:
      continue
    plot_data(example_data, label + " : " + source_id )
    toggles = widgets.ToggleButtons(
      options=['Unset', 'Wing', 'Slope', 'Ring', 'Other'],
      description='Label:',
      disabled=False,
      button_style='success',
      tooltips=['Unset', 'Wing', 'Slope', 'Ring', 'Other'],
    )
    def on_click(change, source_id = source_id):
      global gesture_data
      global hand_labels
      hand_labels.append({"source_id": source_id, "hand_label": change['new']})
    on_click.index = index
    toggles.observe(on_click, 'value')
    display(toggles)


In [10]:

#data_prepare.write_json_file(hand_labels, "hand_labels.json")
#!cat hand_labels.json


## Begin training

The following cell will begin the training process. Training will take around 5 minutes on a GPU runtime. You'll see the metrics in TensorBoard after a few epochs.

In [ ]:
!python train.py --model CNN --data_source v2 --epochs 50

## Create a C source file

The `train.py` script writes a model, `model.tflite`, to the training scripts' directory.

In the following cell, we convert this model into a C++ source file we can use with TensorFlow Lite for Microcontrollers.

In [ ]:
# Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i model.tflite > /content/model.cc
# Print the source file
!cat /content/model.cc